In [11]:
import pandas as pd

# Read data from Excel files
transactions = pd.read_excel("/mnt/d/Downloads_Crome/transactions.xlsx", parse_dates=['date'])
merchants = pd.read_excel("/mnt/d/Downloads_Crome/merchants.xlsx")

# Filter out fraudulent transactions
non_fraud_transactions = transactions[transactions['fraudbits'] == 0]

# Merge transactions with merchants to get the 'coef' and 'guild'
merged_data = pd.merge(non_fraud_transactions, merchants, on='merchantid', how='left')

# Calculate amnesty for each transaction
merged_data['amnesty'] = abs(merged_data['cost'] - merged_data['model_cost']) / merged_data['coef']

# Calculate the number of 'sell' and 'buy' transactions per guild
guild_sell_counts = merged_data[merged_data['countertype'] == 'sell'].groupby('guild').size()
guild_buy_counts = merged_data[merged_data['countertype'] == 'buy'].groupby('guild').size()

# Handle potential division by zero if a guild has no 'buy' transactions
guild_ratio = guild_sell_counts / guild_buy_counts.replace(0, pd.NA)

# Calculate the average amnesty per guild
guild_avg_amnesty = merged_data.groupby('guild')['amnesty'].mean()

# Calculate efficiency per guild
guild_efficiency = guild_ratio * guild_avg_amnesty

# Drop any guilds with NaN efficiency due to division by zero
guild_efficiency = guild_efficiency.dropna()

# Identify the guild with the highest efficiency
most_efficient_guild = guild_efficiency.idxmax()
highest_efficiency_score = guild_efficiency.max()

# Round the efficiency score to the nearest hundredths
rounded_efficiency_score = round(highest_efficiency_score, 2)

# For the most efficient guild, find the hour with the most 'sell' transactions
guild_transactions = merged_data[merged_data['guild'] == most_efficient_guild]
guild_sell_transactions = guild_transactions[guild_transactions['countertype'] == 'sell']

# Extract hour from 'date' field
guild_sell_transactions['hour'] = guild_sell_transactions['date'].dt.hour

# Find the hour with the most 'sell' transactions
if not guild_sell_transactions.empty:
    peak_hour = guild_sell_transactions['hour'].value_counts().idxmax()
else:
    peak_hour = 'N/A'  # Default value if no sell transactions are found

# Output the result
print(f"{most_efficient_guild} {rounded_efficiency_score} {peak_hour}")


/tmp/ipykernel_154051/955201208.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transactions = pd.read_excel("/mnt/d/Downloads_Crome/transactions.xlsx", parse_dates=['date'])


AttributeError: Can only use .dt accessor with datetimelike values

In [12]:
print(transactions.dtypes)

citizenid        int64
cost           float64
countertype     object
date            object
fraudbits        int64
logid            int64
marketid         int64
merchantid       int64
model_cost     float64
dtype: object


In [16]:
print(transactions['date'].head())

0   NaT
1   NaT
2   NaT
3   NaT
4   NaT
Name: date, dtype: datetime64[ns]


In [15]:
transactions['date'] = pd.to_datetime(transactions['date'], format='%Y-%m-%d %H:%M', errors='coerce')

